In [51]:
import pandas as pd
import requests
from pandas import json_normalize
import numpy as np
from io import BytesIO
import time
from datetime import datetime, timedelta
import json
import polars as pl


idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format


# API request config for SSI API endpoints
ssi_headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'X-Fiin-Key': 'KEY',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-Fiin-User-ID': 'ID',
        'X-Fiin-User-Token':'79,14,109,2,77,63,182,226,168,166,146,27,233,245,5,195,0,200,169,49,153,168,150,254,116,248,73,45,23,61,239,77,183,194,66,44,130,28,239,66,27,56,75,64,56,106,32,203,36,15,41,203,250,254,180,226,198,77,152,213,125,234,19,189,68,11,105,241,12,164,235,22,207,252,226,142,142,239,234,206,6,141,63,201,33,169,127,101,9,25,210,180,120,123,145,86,239,76,144,170,24,236,42,224,97,18,41,250,154,193,169,100,133,44,162,88,201,178,212,112,53,114,12,95,81,58,144,233,119,10,24,138,241,254,39,124,10,203,168,121,230,82,103,254,250,60,20,249,237,230,12,76,78,65,234,48,255,62,102,6,201,137,37,132,182,151,246,26,37,81,123,183,82,209,104,37,195,103,138,251,132,30,158,210,151,131,145,67,235,171,34,119,235,73,193,80,232,193,102,189,47,70,108,12,43,253,60,16,114,169,91,125,8,129,40,14,219,191,133,180,200,249,65,81,108,254,163,47,151,3,1,35,132,124,160,200,104,243,64,185,149,215,77,222,174,177,231,227,14,240,127,167,68,133,199,216',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        'X-Fiin-Seed': 'SEED',
        'sec-ch-ua-platform': 'Windows',
        'Origin': 'https://iboard.ssi.com.vn',
        'Sec-Fetch-Site': 'same-site',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://iboard.ssi.com.vn/',
        'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7'
        }


def organ_listing (lang='vi', headers=ssi_headers):
    """
    Return a DataFrame of all available stock symbols. Live data is retrieved from the SSI API.
    Parameters:
        lang (str): language of the data. Default is 'vi', other options are 'en'
        headers (dict): headers of the request
    """
    url = f"https://fiin-core.ssi.com.vn/Master/GetListOrganization?language={lang}"
    response = requests.request("GET", url, headers=headers)
    status = response.status_code
    if status == 200:
        data = response.json()
        # print('Total number of companies: ', data['totalCount'])
        df = pd.DataFrame(data['items'])
        return df
    else:
        print('Error in API response', response.text)
def financial_report (symbol='SSI', report_type='BalanceSheet', frequency='Quarterly', periods=200, latest_year=None, headers=ssi_headers): # Quarterly, Yearly
    """
    Return financial reports of a stock symbol by type and period.
    Args:
        symbol (:obj:`str`, required): 3 digits name of the desired stock.
        report_type (:obj:`str`, required): BalanceSheet, IncomeStatement, CashFlow
        report_range (:obj:`str`, required): Yearly or Quarterly.
    """
    symbol = symbol.upper()
    organ_code = organ_listing().query(f'ticker == @symbol')['organCode'].values[0]
    this_year = str(datetime.now().year)
    if latest_year == None:
      latest_year = this_year
    else:
      if isinstance(latest_year, int) != True:
        print('Please input latest_year as int number')
      else:
        pass
    url = f'https://fiin-fundamental.ssi.com.vn/FinancialStatement/Download{report_type}?language=vi&OrganCode={organ_code}&Skip=0&Frequency={frequency}&numberOfPeriod={periods}&latestYear={latest_year}'
    response = requests.get(url, headers=headers)
    # print(response.text)
    status = response.status_code
    if status == 200:
        df = pd.read_excel(BytesIO(response.content), skiprows=7,engine='openpyxl')
        # .dropna()
        return df
    else:
        print(f'Error {status} when getting data from SSI. Details:\n {response.text}')
        return None

def mc(symbol='SSI',frequency='Quarterly'):
        headers = {
                'Connection': 'keep-alive',
                'sec-ch-ua': '"Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
                'DNT': '1',
                'sec-ch-ua-mobile': '?0',
                'X-Fiin-Key': 'KEY',
                'Content-Type': 'application/json',
                'Accept': 'application/json',
                'X-Fiin-User-ID': 'ID',
                'X-Fiin-Key':'KEY',
                'X-Fiin-Seed':'SEED',
                'X-Fiin-User-Token':'93,203,163,40,224,188,115,115,138,126,18,199,199,124,39,108,231,125,80,15,79,226,178,184,60,101,162,174,35,156,160,54,113,153,99,49,167,98,81,217,225,67,146,16,255,228,25,242,213,192,129,186,139,181,191,112,119,41,36,49,45,37,208,216,184,215,157,52,95,29,185,63,186,228,97,27,86,163,49,131,67,17,92,172,156,132,217,88,15,231,7,175,164,138,29,180,116,130,76,38,107,88,132,186,75,8,124,209,185,88,180,7,211,235,229,42,232,206,219,25,84,76,226,0,197,66,181,79,230,74,208,200,86,229,25,9,26,44,219,167,162,161,178,144,90,239,165,36,41,99,186,205,217,181,7,162,101,238,186,34,56,31,153,19,176,193,110,47,18,237,192,133,113,67,194,227,13,202,239,126,23,189,121,36,77,74,211,188,203,144,113,209,48,248,84,22,237,223,154,232,228,74,124,239,104,86,146,26,237,250,25,53,58,197,214,59,195,119,6,146,131,42,111,254,70,220,1,233,163,205,29,132,94,248,229,177,175,42,32,130,189,20,83,218,153,0,143,234,246,130,103,169,144,137,128,169',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
                'X-Fiin-Seed': 'SEED',
                'sec-ch-ua-platform': 'Windows',
                'Origin': 'https://iboard.ssi.com.vn',
                'Sec-Fetch-Site': 'same-site',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Dest': 'empty',
                'Referer': 'https://iboard.ssi.com.vn/',
                'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7'
                }
        symbol = symbol.upper()
        organ_code = organ_listing().query(f'ticker == @symbol')['organCode'].values[0]
        url = f'https://fiin-fundamental.ssi.com.vn/FinancialAnalysis/DownloadFinancialRatio2?language=vi&OrganCode={organ_code}&CompareToIndustry=false&Frequency={frequency}&Ratios=ryd11&TimeLineFrom=2001&TimeLineTo=2025'
        response = requests.get(url, headers=headers)
        status = response.status_code
        if status == 200:
            df = pd.read_excel(BytesIO(response.content), skiprows=7)
            df = df.rename(columns={'Chỉ số': 'LengthReport'})
            return df.iloc[[1],:]
        else:
            print(f'Error {status} when getting data. Details:\n {response.text}')
            return None



def get_mc_Y(ticker):    
    x = mc(ticker,'Yearly')
    x = x.T
    x.columns = x.iloc[0]
    x = x.iloc[1:,:]
    #include only rows contains ".3" in last 2 string of index
    x = x[x.index.str[-2:] == '.3']
    #get first 4 characters of index
    x.index = x.index.str[:4]
    x['dates'] = x.index.astype(int)
    #replace 0 with NaN value
    x = x.replace(0, np.nan)
    #rename column 2 to 'mc'
    x = x.rename(columns={x.columns[0]: 'mc'})
    return x

In [52]:
def organ_listing (lang='vi', headers=ssi_headers):
    """
    Return a DataFrame of all available stock symbols. Live data is retrieved from the SSI API.
    Parameters:
        lang (str): language of the data. Default is 'vi', other options are 'en'
        headers (dict): headers of the request
    """
    url = f"https://fiin-core.ssi.com.vn/Master/GetListOrganization?language={lang}"
    response = requests.request("GET", url, headers=headers)
    status = response.status_code
    if status == 200:
        data = response.json()
        # print('Total number of companies: ', data['totalCount'])
        df = pd.DataFrame(data['items'])
        return df
    else:
        print('Error in API response', response.text)
        
def get_companyname(ticker,lang): 
    org_listing = organ_listing(lang)
    try:
        org_name = org_listing[org_listing['ticker']==ticker.upper()]['organName']
        org_name = org_name.values[0]
        return org_name
    except:
        return None

#get company industry from ticker, source: SSI ICB industry listing
def get_industry(ticker,lang): 
    try:
        org_listing = organ_listing(lang)
        org_icbCode = org_listing[org_listing['ticker']==ticker.upper()]['icbCode']
        org_icbCode = org_icbCode.values[0]
        url = f'https://fiin-core.ssi.com.vn/Master/GetAllIcbIndustry?language=en'
        response = requests.get(url,headers=ssi_headers)
        response.json()
        df = pd.DataFrame(response.json()['items'])
        industry = df[df['icbCode']==org_icbCode]['icbName'].values[0]
        return industry
    except:
        return None



In [53]:
def get_os(symbol,frequency='Quarterly'): 
    symbol = symbol.upper()
    organ_code = organ_listing().query(f'ticker == @symbol')['organCode'].values[0]
    url = f'https://fiin-fundamental.ssi.com.vn/FinancialAnalysis/DownloadFinancialRatio2?language=vi&OrganCode={organ_code}&CompareToIndustry=false&Frequency={frequency}&Ratios=ryd3&TimeLineFrom=2001&TimeLineTo=2025'
    response = requests.get(url, headers=ssi_headers)
    status = response.status_code
    if status == 200:
        df = pd.read_excel(BytesIO(response.content), skiprows=7)
        df = df.rename(columns={'Chỉ số': 'LengthReport'})
        x = df.T
        x.columns = x.iloc[0]
        x = x.iloc[1:,:]
        x['year'] = x.index.str[-4:].astype(int)
        x['quarter'] = x.index.str[1].astype(float)
        x['dates'] = pd.PeriodIndex(year=x["year"], quarter=x["quarter"])
        x['dates'] = x['dates'].dt.to_timestamp(freq='Q')
        x = x.replace(0, np.nan)
        x = x.rename(columns={x.columns[0]: 'mc'})
        x = x.sort_values(by='dates')
        return x[['year','quarter',symbol]]

    else:
        print(f'Error {status} when getting data. Details:\n {response.text}')
        return None
    
def get_price(ticker, period='Q'):
    url = f'https://s.cafef.vn/Ajax/PageNew/DataHistory/PriceHistory.ashx?Symbol={ticker}&StartDate=&EndDate=&PageIndex=1&PageSize=200000'
    
    # Lấy dữ liệu từ URL
    response = requests.get(url)
    data = response.json()['Data']
    df = pd.DataFrame(data['Data'])
    
    # Tạo cột year, month, date dựa trên cột 'Ngay'
    df[['date', 'month', 'year']] = df['Ngay'].str.split('/', expand=True)
    df[['year', 'month', 'date']] = df[['year', 'month', 'date']].astype(int)
    
    # Tạo cột quarter
    df['quarter'] = df['month'].apply(lambda x: (x-1)//3 + 1)
    
    # Lọc ra ngày giao dịch cuối cùng của mỗi quý
    df_quarters = df[df['month'].isin([3, 6, 9, 12])]
    df_last_days = df_quarters.groupby(['year', 'quarter']).apply(lambda x: x[x['date'] == x['date'].max()])
    
    # Chuyển đổi cột 'GiaDongCua' thành DataFrame và nhân với 10^3
    df_close_price = pd.DataFrame(df_last_days['GiaDongCua'] * (10**3))
    
    # Đặt lại chỉ mục và xóa cột không cần thiết
    df_close_price.reset_index(inplace=True)
    df_close_price.drop(columns=['level_2'], inplace=True)
    df_close_price.rename(columns={'GiaDongCua': 'closePrice'}, inplace=True)

    return df_close_price

def get_mc(symbol, period='Quarterly'):
    df_number_of_shares = get_os(symbol)
    df_close_price = get_price(symbol)
    
    # Kết hợp df_number_of_shares và df_close_price để tạo df_market_cap
    df_market_cap = pd.merge(df_number_of_shares, df_close_price, on=['year', 'quarter'])
    df_market_cap['market_cap'] = df_market_cap[symbol] * df_market_cap['closePrice']
  
    if period == 'Quarterly':
        return df_market_cap
    else:
        # Giữ lại các hàng thuộc quý 4 của mỗi năm
        df_market_cap = df_market_cap[df_market_cap['quarter']==4]
        return df_market_cap



In [54]:
import os
import sys
#import apps.config from folder apps
sys.path.append(os.path.abspath('..'))
from apps.config import *
from apps.fundamental import *
import polars as pl
import pyodbc


list_chitieu = ['TỔNG TÀI SẢN','TÀI SẢN NGẮN HẠN','Tiền và tương đương tiền','Giá trị thuần đầu tư ngắn hạn','Các khoản phải thu','Hàng tồn kho, ròng','TÀI SẢN DÀI HẠN','Phải thu dài hạn','Tài sản cố định','GTCL TSCĐ hữu hình','Nguyên giá TSCĐ hữu hình','Khấu hao lũy kế TSCĐ hữu hình','GTCL Tài sản thuê tài chính','Nguyên giá tài sản thuê tài chính','Khấu hao lũy kế tài sản thuê tài chính','GTCL tài sản cố định vô hình','Nguyên giá TSCĐ vô hình','Khấu hao lũy kế TSCĐ vô hình','Bất động sản đầu tư','Nguyên giá tài sản đầu tư','Khấu hao lũy kế tài sản đầu tư','Tài sản dở dang dài hạn','Đầu tư dài hạn',
                'NỢ PHẢI TRẢ','Nợ ngắn hạn','Phải trả người bán','Người mua trả tiền trước','Doanh thu chưa thực hiện ngắn hạn','Vay ngắn hạn','Nợ dài hạn','Người mua trả tiền trước dài hạn','Doanh thu chưa thực hiên','Vay dài hạn','Trái phiếu chuyển đổi','VỐN CHỦ SỞ HỮU','Vốn góp','Thặng dư vốn cổ phần','Cổ phiếu Quỹ','Lãi chưa phân phối','Lợi ích cổ đông không kiểm soát','Doanh số thuần','Lãi gộp','Thu nhập tài chính','Chi phí tài chính','Trong đó: Chi phí lãi vay','Lãi/(lỗ) từ công ty liên doanh','Chi phí bán hàng','Chi phí quản lý doanh  nghiệp','Thu nhập khác, ròng','Lãi/(lỗ) ròng trước thuế','Lãi/(lỗ) thuần sau thuế','Lợi nhuận của Cổ đông của Công ty mẹ',
                'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh','Khấu hao TSCĐ',
                # 'Chi phí dự phòng','Chi phí lãi vay','Chi phí lãi vay đã trả','Thuế thu nhập doanh nghiệp đã trả',
                'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư','Tiền mua tài sản cố định và các tài sản dài hạn khác','Tiền thu được từ thanh lý tài sản cố định','Cổ tức và tiền lãi nhận được','Lưu chuyển tiền tệ từ hoạt động tài chính','Tiền thu từ phát hành cổ phiếu và vốn góp','Chi trả cho việc mua lại, trả lại cổ phiếu','Tiền thu được các khoản đi vay','Tiển trả các khoản đi vay','Tiền thanh toán vốn gốc đi thuê tài chính','Cổ tức đã trả'
]
def add_ratios_Q(df):
    df = df[list_chitieu]
    
    df['bs_cash'] = df.loc[:, 'Tiền và tương đương tiền'] + df.loc[:, 'Giá trị thuần đầu tư ngắn hạn']
    df['bs_ar'] = df.loc[:, 'Các khoản phải thu'] + df.loc[:, 'Phải thu dài hạn']
    df['bs_fa'] = df.loc[:, 'GTCL TSCĐ hữu hình'] + df.loc[:, 'GTCL Tài sản thuê tài chính'] + df.loc[:, 'GTCL tài sản cố định vô hình'] + df.loc[:, 'Bất động sản đầu tư']
    df['bs_gross_fa'] = df.loc[:, 'Nguyên giá TSCĐ hữu hình'] + df.loc[:, 'Nguyên giá tài sản thuê tài chính'] + df.loc[:, 'Nguyên giá TSCĐ vô hình'] + df.loc[:, 'Nguyên giá tài sản đầu tư']
    df['other_asset'] = df.loc[:, 'TỔNG TÀI SẢN'] - df.loc[:, 'bs_cash'] - df.loc[:, 'bs_ar'] - df.loc[:, 'Hàng tồn kho, ròng'] - df.loc[:, 'bs_fa'] - df.loc[:, 'Tài sản dở dang dài hạn'] - df.loc[:, 'Đầu tư dài hạn']
    df['bs_cust_pre'] = df.loc[:, 'Doanh thu chưa thực hiện ngắn hạn'] + df.loc[:, 'Doanh thu chưa thực hiên'] + df.loc[:, 'Người mua trả tiền trước'] + df.loc[:, 'Người mua trả tiền trước dài hạn']
    df['debt'] = df.loc[:, 'Vay ngắn hạn'] + df.loc[:, 'Vay dài hạn'] + df.loc[:, 'Trái phiếu chuyển đổi']
    df['other_lia'] = df.loc[:, 'NỢ PHẢI TRẢ'] - df.loc[:, 'debt'] - df.loc[:, 'Phải trả người bán'] - df.loc[:, 'bs_cust_pre']
    df['other_equity'] = df.loc[:, 'VỐN CHỦ SỞ HỮU'] - df.loc[:, 'Vốn góp'] - df.loc[:, 'Lãi chưa phân phối'] - df.loc[:, 'Cổ phiếu Quỹ']
    df['netdebt'] = df.loc[:, 'debt'] - df.loc[:, 'bs_cash']
    df['ca/ta'] = df.loc[:, 'TÀI SẢN NGẮN HẠN'] / (1 + df.loc[:, 'TỔNG TÀI SẢN'])
    df['de'] = df.loc[:, 'debt'] / (1 + df.loc[:, 'VỐN CHỦ SỞ HỮU'])
    
    df['tax_rate'] = 1 - (df.loc[:, 'Lãi/(lỗ) thuần sau thuế'] / (1 + df.loc[:, 'Lãi/(lỗ) ròng trước thuế']))
    df['op'] = df.loc[:, 'Lãi gộp'] + df.loc[:, 'Chi phí bán hàng'] + df.loc[:, 'Chi phí quản lý doanh  nghiệp']
    df['core_e'] = df.loc[:, 'op'] * (1 - df.loc[:, 'tax_rate'])
    df['fin_income'] = df.loc[:, 'Thu nhập tài chính'] + (df.loc[:, 'Chi phí tài chính'] - df.loc[:, 'Trong đó: Chi phí lãi vay'])
    df['EBT'] = df.loc[:, 'op'] + df.loc[:, 'Trong đó: Chi phí lãi vay']
    
    df['cf_div'] = df.loc[:, 'Cổ tức đã trả'] + df.loc[:, 'Chi trả cho việc mua lại, trả lại cổ phiếu']
    df['cf_delta_debt'] = df.loc[:, 'Tiền thu được các khoản đi vay'] + df.loc[:, 'Tiển trả các khoản đi vay'] + df.loc[:, 'Tiền thanh toán vốn gốc đi thuê tài chính']
    df['cf_dep'] = df.loc[:, 'Khấu hao TSCĐ']
    df['cf_khac'] = (df.loc[:, 'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh'] + df.loc[:, 'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư'] + df.loc[:, 'Lưu chuyển tiền tệ từ hoạt động tài chính']) - (df.loc[:, 'Lãi/(lỗ) thuần sau thuế'] + df.loc[:, 'cf_dep'] + df.loc[:, 'Tiền mua tài sản cố định và các tài sản dài hạn khác'] + df.loc[:, 'Tiền thu từ phát hành cổ phiếu và vốn góp'] + df.loc[:, 'cf_delta_debt'] + df.loc[:, 'cf_div'])
    
    df['operating_EBITDA'] = df.loc[:, 'op'] + df.loc[:, 'cf_dep']
    df['EBITDA'] = df.loc[:, 'Lãi/(lỗ) ròng trước thuế'] - df.loc[:, 'Thu nhập khác, ròng'] + df.loc[:, 'cf_dep'] - df.loc[:, 'Trong đó: Chi phí lãi vay']
    
    return df

col1 = ['Lãi gộp', 'op', 'EBT', 'Lãi/(lỗ) ròng trước thuế', 'Lãi/(lỗ) thuần sau thuế', 'Lợi nhuận của Cổ đông của Công ty mẹ', 'core_e','EBITDA']
col3 = ['Doanh số thuần', 'Lãi gộp', 'op', 'EBT', 'Lãi/(lỗ) ròng trước thuế', 'Lãi/(lỗ) thuần sau thuế', 'Lợi nhuận của Cổ đông của Công ty mẹ', 'core_e','EBITDA']
col2 = ['Doanh số thuần', 'Lãi gộp', 'op', 'EBT', 'Lãi/(lỗ) ròng trước thuế', 'Lãi/(lỗ) thuần sau thuế', 'Lợi nhuận của Cổ đông của Công ty mẹ', 'core_e','EBITDA', 
        'Doanh số thuần_4Q', 'Lãi gộp_4Q', 'op_4Q','Lãi/(lỗ) ròng trước thuế', 'Lãi/(lỗ) thuần sau thuế_4Q', 'Lợi nhuận của Cổ đông của Công ty mẹ_4Q', 'core_e_4Q','EBITDA_4Q']
def margin_func(x):
    for i in col1:
        x[i + "_m"] = x[i] / (1+x['Doanh số thuần'])
    return x

def ttm(x):
    for i in col3:
        x[i + "_4Q"] = x[i].rolling(4, min_periods=4).sum()
    return x

def g_func(x):
    for i in col2:
        x["g_" + i] = x[i].pct_change(periods=3) 
    x['roe_4Q'] = x['Lãi/(lỗ) thuần sau thuế_4Q']/x['VỐN CHỦ SỞ HỮU'].rolling(4, min_periods=4).mean()
    x['roe_core_4Q'] = x['core_e_4Q']/x['VỐN CHỦ SỞ HỮU'].rolling(4, min_periods=4).mean()
    x['roa_4Q'] = x['Lãi/(lỗ) thuần sau thuế_4Q']/x['TỔNG TÀI SẢN'].rolling(4, min_periods=4).mean()
    return x

def get_fs_Q(ticker):
    bs = financial_report(ticker,'BalanceSheet','Quarterly')
    # bs = bs.loc[:, (bs==0).mean() < .6]
    pl = financial_report(ticker,'IncomeStatement','Quarterly')
    # pl = pl.loc[:, (pl==0).mean() < .6]
    cf = financial_report(ticker,'CashFlow','Quarterly')
    cf = cf.rename(columns={'Unnamed: 0': 'CHỈ TIÊU'})
    cf.set_index('CHỈ TIÊU', inplace=True)
    cf2 = pd.DataFrame(index = {'Khấu hao TSCĐ':'CHỈ TIÊU'}, columns = cf.columns).fillna(0)
    if cf2.index[0] in cf.index:
        pass
    else:
        cf = pd.concat([cf,cf2],axis=0)
    cf = cf.reset_index().rename(columns={'index': 'CHỈ TIÊU'})

    fs = pd.concat([bs,pl,cf])
    #delete all the column with NaN value > 40
    fs = fs.dropna(axis=1,thresh=40)
    fs = fs.T
    fs.columns = fs.iloc[0]
    fs = fs.iloc[1:,:]
    #Dropping rows if more than half of the values are zeros 
    # fs = fs.loc[fs.isna().sum(axis=1)<50]

    fs = add_ratios_Q(fs)
    fs = margin_func(fs)
    fs = ttm(fs)
    fs = g_func(fs)
    fs['year'] = fs.index.str[-4:].astype(int)
    fs['quarter'] = fs.index.str[1].astype(float)
    fs['dates'] = pd.PeriodIndex(year=fs["year"], quarter=fs["quarter"])
    fs['dates'] = fs['dates'].dt.to_timestamp(freq='Q')
    fs = fs.sort_values(by='dates')
    return fs

def get_data_Q(ticker):
    x = get_fs_Q(ticker)
    x = x.reset_index()
    x=pl.from_pandas(x)
    x = x.with_columns(pl.col('quarter').cast(pl.Int64))
    y = get_mc(ticker,period='Quarterly')
    y=pl.from_pandas(y)
    merged_df = x.join(y, on=['year','quarter'], how='inner')
    merged_df = merged_df.with_columns((pl.col('marketCap')/pl.col('Lãi/(lỗ) thuần sau thuế_4Q')).alias('P/E'))
    merged_df = merged_df.with_columns((pl.col('marketCap')/pl.col('VỐN CHỦ SỞ HỮU')).alias('P/B'))
    merged_df = merged_df.with_columns([(pl.col('dates').dt.strftime("%Y")).alias('dates')])
    return merged_df

In [55]:
get_data_Q('CNG')

C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_1612\2418902246.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bs_cash'] = df.loc[:, 'Tiền và tương đương tiền'] + df.loc[:, 'Giá trị thuần đầu tư ngắn hạn']
C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_1612\2418902246.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bs_ar'] = df.loc[:, 'Các khoản phải thu'] + df.loc[:, 'Phải thu dài hạn']
C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_1612\2418902246.py:22: SettingWithCopyWarning: 

CHỈ TIÊU,TỔNG TÀI SẢN,TÀI SẢN NGẮN HẠN,Tiền và tương đương tiền,Giá trị thuần đầu tư ngắn hạn,Các khoản phải thu,"Hàng tồn kho, ròng",TÀI SẢN DÀI HẠN,Phải thu dài hạn,Tài sản cố định,GTCL TSCĐ hữu hình,...,g_Lãi/(lỗ) thuần sau thuế_4Q,g_Lợi nhuận của Cổ đông của Công ty mẹ_4Q,g_core_e_4Q,g_EBITDA_4Q,roe_4Q,roe_core_4Q,roa_4Q,year,quarter,dates
Q2 2011,"598,812,814,848.000","286,512,941,096.000","75,312,486,151.000","63,100,000,000.000","135,507,832,009.000","7,867,373,579.000","312,299,873,752.000",0.000,"298,739,331,191.000","274,109,306,422.000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,2.000,2011-06-30
Q3 2011,"632,941,426,858.000","329,790,779,866.000","131,577,454,935.000","79,600,000,000.000","109,887,995,675.000","7,310,908,042.000","303,150,646,992.000",0.000,"284,895,904,931.000","284,665,621,792.000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,3.000,2011-09-30
Q4 2011,"707,835,556,405.000","398,943,373,276.000","179,293,614,903.000","63,100,000,000.000","138,329,685,044.000","10,005,661,391.000","308,892,183,129.000",0.000,"292,344,422,568.000","248,982,639,215.000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,4.000,2011-12-31
Q1 2012,"654,981,645,305.000","373,404,797,672.000","168,889,440,097.000","62,550,000,000.000","120,956,972,222.000","16,344,441,718.000","281,576,847,633.000",0.000,"265,510,166,165.000","225,525,178,249.000",...,NaN,NaN,NaN,NaN,0.493,0.526,0.291,2012,1.000,2012-03-31
Q2 2012,"592,599,207,522.000","281,726,418,229.000","113,204,095,865.000","15,100,000,000.000","134,616,722,669.000","16,353,590,898.000","310,872,789,293.000",0.000,"234,806,107,825.000","198,736,388,932.000",...,NaN,NaN,NaN,NaN,0.388,0.412,0.234,2012,2.000,2012-06-30
Q3 2012,"622,177,203,384.000","337,557,909,475.000","129,717,546,756.000","30,100,000,000.000","153,209,790,879.000","21,933,991,349.000","284,619,293,909.000",0.000,"208,552,612,441.000","173,259,131,406.000",...,NaN,NaN,NaN,NaN,0.298,0.318,0.180,2012,3.000,2012-09-30
Q4 2012,"623,204,458,607.000","315,890,255,598.000","85,305,119,116.000","55,100,000,000.000","152,125,223,262.000","21,500,912,441.000","307,314,203,009.000",0.000,"225,197,521,552.000","191,757,562,055.000",...,-0.374,-0.374,-0.387,-0.192,0.305,0.319,0.190,2012,4.000,2012-12-31
Q1 2013,"590,829,662,049.000","318,745,860,636.000","115,055,387,722.000","35,100,000,000.000","141,425,296,981.000","21,462,443,283.000","272,083,801,413.000",0.000,"192,384,896,968.000","167,824,583,401.000",...,-0.209,-0.209,-0.220,-0.114,0.319,0.334,0.197,2013,1.000,2013-03-31
Q2 2013,"611,403,729,332.000","373,482,014,646.000","192,588,487,112.000","7,000,000,000.000","147,730,123,096.000","21,274,762,375.000","237,921,714,686.000",0.000,"158,222,810,241.000","137,181,068,724.000",...,0.028,0.028,-0.028,-0.006,0.314,0.317,0.195,2013,2.000,2013-06-30
Q3 2013,"637,556,005,881.000","425,483,453,806.000","208,297,956,738.000","12,000,000,000.000","178,692,539,811.000","21,962,499,315.000","212,072,552,075.000",0.000,"132,373,647,630.000","107,002,398,163.000",...,0.026,0.026,-0.024,0.027,0.316,0.314,0.197,2013,3.000,2013-09-30


In [39]:
x=  get_data_Q('VNM')
x = x.write_json(row_oriented=True)
chart = pl.from_pandas(pd.read_json(x))
date = chart.select(pl.col("dates")).to_series()
date

dates
i64
2006
2006
2006
2006
2007
2007
2007
2007
2008


In [13]:
data = {
    'Tiền và tương đương tiền': [100]*1000000,
    'Giá trị thuần đầu tư ngắn hạn': [200]*1000000,
    'Các khoản phải thu': [300]*1000000,
    'Phải thu dài hạn': [400]*1000000,
    'GTCL TSCĐ hữu hình': [500]*1000000,
    'GTCL Tài sản thuê tài chính': [600]*1000000,
    'GTCL tài sản cố định vô hình': [700]*1000000,
    'Bất động sản đầu tư': [800]*1000000,
    'Nguyên giá TSCĐ hữu hình': [900]*1000000,
    'Nguyên giá tài sản thuê tài chính': [1000]*1000000,
    'Nguyên giá TSCĐ vô hình': [1100]*1000000,
    'Nguyên giá tài sản đầu tư': [1200]*1000000,
    'TỔNG TÀI SẢN': [1300]*1000000,
    'Hàng tồn kho, ròng': [1400]*1000000,
    'Tài sản dở dang dài hạn': [1500]*1000000,
    'Đầu tư dài hạn': [1600]*1000000,
    'Doanh thu chưa thực hiện ngắn hạn': [1700]*1000000,
    'Doanh thu chưa thực hiên': [1800]*1000000,
    'Người mua trả tiền trước': [1900]*1000000,
    'Người mua trả tiền trước dài hạn': [2000]*1000000,
    'Vay ngắn hạn': [2100]*1000000,
    'Vay dài hạn': [2200]*1000000,
    'Trái phiếu chuyển đổi': [2300]*1000000,
    'NỢ PHẢI TRẢ': [2400]*1000000,
    'Phải trả người bán': [2500]*1000000,
    'VỐN CHỦ SỞ HỮU': [2600]*1000000,
    'Vốn góp': [2700]*1000000,
    'Lãi chưa phân phối': [2800]*1000000,
    'Cổ phiếu Quỹ': [2900]*1000000,
    'Lãi/(lỗ) thuần sau thuế': [3000]*1000000,
    'Lãi/(lỗ) ròng trước thuế': [3100]*1000000,
    'Lãi gộp': [3200]*1000000,
    'Chi phí bán hàng': [3300]*1000000,
    'Chi phí quản lý doanh nghiệp': [3400]*1000000,
    'Thu nhập tài chính': [3500]*1000000,
    'Chi phí tài chính': [3600]*1000000,
    'Trong đó: Chi phí lãi vay': [3700]*1000000,
    'Cổ tức đã trả': [3800]*1000000,
    'Chi trả cho việc mua lại, trả lại cổ phiếu': [3900]*1000000,
    'Tiền thu được các khoản đi vay': [4000]*1000000,
    'Tiển trả các khoản đi vay': [4100]*1000000,
    'Tiền thanh toán vốn gốc đi thuê tài chính': [4200]*1000000,
    'Khấu hao TSCĐ': [4300]*1000000,
    'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh': [4400]*1000000,
    'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư': [4500]*1000000,
    'Lưu chuyển tiền tệ từ hoạt động tài chính': [4600]*1000000,
    'Tiền mua tài sản cố định và các tài sản dài hạn khác': [4700]*1000000,
    'Tiền thu từ phát hành cổ phiếu và vốn góp': [4800]*1000000
}

In [16]:
def add_ratios_Q_pandas(df):    
    df['bs_cash'] = df.loc[:, 'Tiền và tương đương tiền'] + df.loc[:, 'Giá trị thuần đầu tư ngắn hạn']
    df['bs_ar'] = df.loc[:, 'Các khoản phải thu'] + df.loc[:, 'Phải thu dài hạn']
    df['bs_fa'] = df.loc[:, 'GTCL TSCĐ hữu hình'] + df.loc[:, 'GTCL Tài sản thuê tài chính'] + df.loc[:, 'GTCL tài sản cố định vô hình'] + df.loc[:, 'Bất động sản đầu tư']
    df['bs_gross_fa'] = df.loc[:, 'Nguyên giá TSCĐ hữu hình'] + df.loc[:, 'Nguyên giá tài sản thuê tài chính'] + df.loc[:, 'Nguyên giá TSCĐ vô hình'] + df.loc[:, 'Nguyên giá tài sản đầu tư']
    df['other_asset'] = df.loc[:, 'TỔNG TÀI SẢN'] - df.loc[:, 'bs_cash'] - df.loc[:, 'bs_ar'] - df.loc[:, 'Hàng tồn kho, ròng'] - df.loc[:, 'bs_fa'] - df.loc[:, 'Tài sản dở dang dài hạn'] - df.loc[:, 'Đầu tư dài hạn']
    df['bs_cust_pre'] = df.loc[:, 'Doanh thu chưa thực hiện ngắn hạn'] + df.loc[:, 'Doanh thu chưa thực hiên'] + df.loc[:, 'Người mua trả tiền trước'] + df.loc[:, 'Người mua trả tiền trước dài hạn']
    df['debt'] = df.loc[:, 'Vay ngắn hạn'] + df.loc[:, 'Vay dài hạn'] + df.loc[:, 'Trái phiếu chuyển đổi']
    df['other_lia'] = df.loc[:, 'NỢ PHẢI TRẢ'] - df.loc[:, 'debt'] - df.loc[:, 'Phải trả người bán'] - df.loc[:, 'bs_cust_pre']
    df['other_equity'] = df.loc[:, 'VỐN CHỦ SỞ HỮU'] - df.loc[:, 'Vốn góp'] - df.loc[:, 'Lãi chưa phân phối'] - df.loc[:, 'Cổ phiếu Quỹ']
    df['netdebt'] = df.loc[:, 'debt'] - df.loc[:, 'bs_cash']
    df['ca/ta'] = df.loc[:, 'TÀI SẢN NGẮN HẠN'] / (1 + df.loc[:, 'TỔNG TÀI SẢN'])
    df['de'] = df.loc[:, 'debt'] / (1 + df.loc[:, 'VỐN CHỦ SỞ HỮU'])
    
    df['tax_rate'] = 1 - (df.loc[:, 'Lãi/(lỗ) thuần sau thuế'] / (1 + df.loc[:, 'Lãi/(lỗ) ròng trước thuế']))
    df['op'] = df.loc[:, 'Lãi gộp'] + df.loc[:, 'Chi phí bán hàng'] + df.loc[:, 'Chi phí quản lý doanh nghiệp']
    df['core_e'] = df.loc[:, 'op'] * (1 - df.loc[:, 'tax_rate'])
    df['fin_income'] = df.loc[:, 'Thu nhập tài chính'] + (df.loc[:, 'Chi phí tài chính'] - df.loc[:, 'Trong đó: Chi phí lãi vay'])
    df['EBT'] = df.loc[:, 'op'] + df.loc[:, 'Trong đó: Chi phí lãi vay']
    
    df['cf_div'] = df.loc[:, 'Cổ tức đã trả'] + df.loc[:, 'Chi trả cho việc mua lại, trả lại cổ phiếu']
    df['cf_delta_debt'] = df.loc[:, 'Tiền thu được các khoản đi vay'] + df.loc[:, 'Tiển trả các khoản đi vay'] + df.loc[:, 'Tiền thanh toán vốn gốc đi thuê tài chính']
    df['cf_dep'] = df.loc[:, 'Khấu hao TSCĐ']
    df['cf_khac'] = (df.loc[:, 'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh'] + df.loc[:, 'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư'] + df.loc[:, 'Lưu chuyển tiền tệ từ hoạt động tài chính']) - (df.loc[:, 'Lãi/(lỗ) thuần sau thuế'] + df.loc[:, 'cf_dep'] + df.loc[:, 'Tiền mua tài sản cố định và các tài sản dài hạn khác'] + df.loc[:, 'Tiền thu từ phát hành cổ phiếu và vốn góp'] + df.loc[:, 'cf_delta_debt'] + df.loc[:, 'cf_div'])
    
    df['operating_EBITDA'] = df.loc[:, 'op'] + df.loc[:, 'cf_dep']
    df['EBITDA'] = df.loc[:, 'Lãi/(lỗ) ròng trước thuế'] - df.loc[:, 'Thu nhập khác, ròng'] + df.loc[:, 'cf_dep'] - df.loc[:, 'Trong đó: Chi phí lãi vay']
    
    return df


In [17]:
import time
# Giả lập dữ liệu đầu vào
df = pd.DataFrame(data)
# Hàm add_ratios_Q với pandas
start_time = time.time()
df_pandas = add_ratios_Q_pandas(df)
end_time = time.time()
print(f"Thời gian chạy với pandas: {end_time - start_time} giây")


KeyError: 'TÀI SẢN NGẮN HẠN'

In [9]:
import polars as pl
def add_ratios_Q_polars(df):
    df = df.with_columns([
        (pl.col('Tiền và tương đương tiền') + pl.col('Giá trị thuần đầu tư ngắn hạn')).alias('bs_cash'),
        (pl.col('Các khoản phải thu') + pl.col('Phải thu dài hạn')).alias('bs_ar'),
        (pl.col('GTCL TSCĐ hữu hình') + pl.col('GTCL Tài sản thuê tài chính') + pl.col('GTCL tài sản cố định vô hình') + pl.col('Bất động sản đầu tư')).alias('bs_fa'),
        (pl.col('Nguyên giá TSCĐ hữu hình') + pl.col('Nguyên giá tài sản thuê tài chính') + pl.col('Nguyên giá TSCĐ vô hình') + pl.col('Nguyên giá tài sản đầu tư')).alias('bs_gross_fa'),
        (pl.col('TỔNG TÀI SẢN') - pl.col('bs_cash') - pl.col('bs_ar') - pl.col('Hàng tồn kho, ròng') - pl.col('bs_fa') - pl.col('Tài sản dở dang dài hạn') - pl.col('Đầu tư dài hạn')).alias('other_asset'),
        (pl.col('Doanh thu chưa thực hiện ngắn hạn') + pl.col('Doanh thu chưa thực hiên') + pl.col('Người mua trả tiền trước') + pl.col('Người mua trả tiền trước dài hạn')).alias('bs_cust_pre'),
        (pl.col('Vay ngắn hạn') + pl.col('Vay dài hạn') + pl.col('Trái phiếu chuyển đổi')).alias('debt'),
        (pl.col('NỢ PHẢI TRẢ') - pl.col('debt') - pl.col('Phải trả người bán') - pl.col('bs_cust_pre')).alias('other_lia'),
        (pl.col('VỐN CHỦ SỞ HỮU') - pl.col('Vốn góp') - pl.col('Lãi chưa phân phối') - pl.col('Cổ phiếu Quỹ')).alias('other_equity'),
        (pl.col('debt') - pl.col('bs_cash')).alias('netdebt'),
        (pl.col('TÀI SẢN NGẮN HẠN') / (1 + pl.col('TỔNG TÀI SẢN'))).alias('ca/ta'),
        (pl.col('debt') / (1 + pl.col('VỐN CHỦ SỞ HỮU'))).alias('de'),
        (1 - (pl.col('Lãi/(lỗ) thuần sau thuế') / (1 + pl.col('Lãi/(lỗ) ròng trước thuế')))).alias('tax_rate'),
        (pl.col('Lãi gộp') + pl.col('Chi phí bán hàng') + pl.col('Chi phí quản lý doanh nghiệp')).alias('op'),
        (pl.col('op') * (1 - pl.col('tax_rate'))).alias('core_e'),
        (pl.col('Thu nhập tài chính') + (pl.col('Chi phí tài chính') - pl.col('Trong đó: Chi phí lãi vay'))).alias('fin_income'),
        (pl.col('op') + pl.col('Trong đó: Chi phí lãi vay')).alias('EBT'),
        (pl.col('Cổ tức đã trả') + pl.col('Chi trả cho việc mua lại, trả lại cổ phiếu')).alias('cf_div'),
        (pl.col('Tiền thu được các khoản đi vay') + pl.col('Tiển trả các khoản đi vay') + pl.col('Tiền thanh toán vốn gốc đi thuê tài chính')).alias('cf_delta_debt'),
        (pl.col('Khấu hao TSCĐ')).alias('cf_dep'),
        ((pl.col('Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh') + pl.col('Lưu chuyển tiền tệ ròng từ hoạt động đầu tư') + pl.col('Lưu chuyển tiền tệ từ hoạt động tài chính')) - (pl.col('Lãi/(lỗ) thuần sau thuế') + pl.col('cf_dep') + pl.col('Tiền mua tài sản cố định và các tài sản dài hạn khác') + pl.col('Tiền thu từ phát hành cổ phiếu và vốn góp') + pl.col('cf_delta_debt') + pl.col('cf_div'))).alias('cf_khac'),
        (pl.col('op') + pl.col('cf_dep')).alias('operating_EBITDA'),
        ((pl.col('Lãi/(lỗ) ròng trước thuế') - pl.col('Thu nhập khác, ròng') + pl.col('cf_dep') - pl.col('Trong đó: Chi phí lãi vay'))).alias('EBITDA')
    ])
    return df


In [10]:
import time

# Giả lập dữ liệu đầu vào
data = {
    'Tiền và tương đương tiền': [100]*1000000,
    'Giá trị thuần đầu tư ngắn hạn': [200]*1000000,
    'Các khoản phải thu': [300]*1000000,
    'Phải thu dài hạn': [400]*1000000,
    'GTCL TSCĐ hữu hình': [500]*1000000,
    'GTCL Tài sản thuê tài chính': [600]*1000000,
    'GTCL tài sản cố định vô hình': [700]*1000000,
    'Bất động sản đầu tư': [800]*1000000,
    'Nguyên giá TSCĐ hữu hình': [900]*1000000,
    'Nguyên giá tài sản thuê tài chính': [1000]*1000000,
    'Nguyên giá TSCĐ vô hình': [1100]*1000000,
    'Nguyên giá tài sản đầu tư': [1200]*1000000,
    'TỔNG TÀI SẢN': [1300]*1000000,
    'Hàng tồn kho, ròng': [1400]*1000000,
    'Tài sản dở dang dài hạn': [1500]*1000000,
    'Đầu tư dài hạn': [1600]*1000000,
    'Doanh thu chưa thực hiện ngắn hạn': [1700]*1000000,
    'Doanh thu chưa thực hiên': [1800]*1000000,
    'Người mua trả tiền trước': [1900]*1000000,
    'Người mua trả tiền trước dài hạn': [2000]*1000000,
    'Vay ngắn hạn': [2100]*1000000,
    'Vay dài hạn': [2200]*1000000,
    'Trái phiếu chuyển đổi': [2300]*1000000,
    'NỢ PHẢI TRẢ': [2400]*1000000,
    'Phải trả người bán': [2500]*1000000,
    'VỐN CHỦ SỞ HỮU': [2600]*1000000,
    'Vốn góp': [2700]*1000000,
    'Lãi chưa phân phối': [2800]*1000000,
    'Cổ phiếu Quỹ': [2900]*1000000,
    'Lãi/(lỗ) thuần sau thuế': [3000]*1000000,
    'Lãi/(lỗ) ròng trước thuế': [3100]*1000000,
    'Lãi gộp': [3200]*1000000,
    'Chi phí bán hàng': [3300]*1000000,
    'Chi phí quản lý doanh nghiệp': [3400]*1000000,
    'Thu nhập tài chính': [3500]*1000000,
    'Chi phí tài chính': [3600]*1000000,
    'Trong đó: Chi phí lãi vay': [3700]*1000000,
    'Cổ tức đã trả': [3800]*1000000,
    'Chi trả cho việc mua lại, trả lại cổ phiếu': [3900]*1000000,
    'Tiền thu được các khoản đi vay': [4000]*1000000,
    'Tiển trả các khoản đi vay': [4100]*1000000,
    'Tiền thanh toán vốn gốc đi thuê tài chính': [4200]*1000000,
    'Khấu hao TSCĐ': [4300]*1000000,
    'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh': [4400]*1000000,
    'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư': [4500]*1000000,
    'Lưu chuyển tiền tệ từ hoạt động tài chính': [4600]*1000000,
    'Tiền mua tài sản cố định và các tài sản dài hạn khác': [4700]*1000000,
    'Tiền thu từ phát hành cổ phiếu và vốn góp': [4800]*1000000
}

df = pl.DataFrame(data)

# Hàm add_ratios_Q với polars
start_time = time.time()
df_polars = add_ratios_Q_polars(df)
end_time = time.time()
print(f"Thời gian chạy với polars: {end_time - start_time} giây")


ColumnNotFoundError: bs_cash

Error originated just after this operation:
DF ["Tiền và tương đương tiền", "Giá trị thuần đầu tư ngắn hạn", "Các khoản phải thu", "Phải thu dài hạn"]; PROJECT */48 COLUMNS; SELECTION: "None"